### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
model_data.shape

(47632, 2832)

In [5]:
model_data.shape

(47632, 2832)

### Drop Null target records

In [6]:
model_data['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [7]:
df = model_data.dropna(subset=['G15_B24'])

In [8]:
df['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [10]:
# take one copy for model development data
data = df.copy()
#Handling infinity and replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [11]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]

In [12]:
model_df1.shape, oot_df1.shape

((34344, 2832), (10802, 2832))

In [13]:
 features_list = [
'TIME_SINCE_FIRST(raw.created_timestamp WHERE hour_part2 = 1)#ever',
'NUM_UNIQUE(pg_success_raw_data.YEAR(created_time_ist))',
'days_since_first_premium_bank_trans',
'NUM_UNIQUE(pg_success_raw_data.bank_title)',
'MEAN(raw.amount WHERE mode_cleaned = 2)#ever',
'max_one_time_spent',
'AVG_TIME_BETWEEN(raw.created_timestamp WHERE mode_cleaned = 2)#ever',
'distinct_merchant',
'days_since_first_ecom_trans',
'days_on_pg',
   
     
'month_year',
'G15_B24']

In [14]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [15]:
oot_df = oot_df1[features_list]

In [16]:
model_df.shape, oot_df.shape

((34344, 12), (10802, 12))

In [17]:
model_df['month_year'].isnull().sum()

np.int64(0)

In [18]:
#Handling Null Values
for i in model_df.columns:
    model_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3044630/2889689820.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  model_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3044630/2889689820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[i].fillna(-9999,inplace=True)


In [19]:
#Handling Null Values
for i in oot_df.columns:
    oot_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3044630/1074864086.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  oot_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3044630/1074864086.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oot_df[i].fillna(-9999,inplace=True)


In [20]:
model_df['G15_B24'].value_counts(dropna=False)

G15_B24
0    32193
1     2151
Name: count, dtype: int64

In [21]:
oot_df['G15_B24'].value_counts(dropna=False)

G15_B24
0    9980
1     822
Name: count, dtype: int64

In [22]:
#Takeout record where banking data only present(exclude -9999)
model_df = model_df[(model_df !=-9999).all(axis=1)]

In [23]:
model_df.shape

(11256, 12)

In [24]:
oot_df = oot_df[(oot_df !=-9999).all(axis=1)]


In [25]:
oot_df.shape

(2889, 12)

In [26]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

In [27]:
X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

In [28]:
X_oot.shape,y_oot.shape

((2889, 10), (2889,))

In [29]:
X_model.shape

(11256, 10)

### Split Model dataset into Train and Test

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [31]:
X_train.shape, X_test.shape

((6753, 10), (4503, 10))

In [32]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(5402, 10) (1351, 10) (4503, 10)


In [33]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)

In [34]:

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.1,
    'max_depth': 2,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'lambda': 10,
    'alpha': 7,
    'gamma': 6.6,
    'scale_pos_weight': 1,
    'random_state': 24   
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 80,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.50000	eval-auc:0.50000
[10]	train-auc:0.54639	eval-auc:0.52351
[20]	train-auc:0.54639	eval-auc:0.52351
[30]	train-auc:0.55837	eval-auc:0.52297
[40]	train-auc:0.55837	eval-auc:0.52297
[43]	train-auc:0.55837	eval-auc:0.52297


In [35]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.56, Gini: 0.12, KS: 0.10
Eval AUC: 0.52, Gini: 0.05, KS: 0.05
Test AUC: 0.54, Gini: 0.09, KS: 0.08
OOT AUC: 0.54, Gini: 0.08, KS: 0.07


In [36]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                             Feature  Importance
0  TIME_SINCE_FIRST(raw.created_timestamp WHERE h...         2.0
1       MEAN(raw.amount WHERE mode_cleaned = 2)#ever         1.0
2                                         days_on_pg         1.0


In [37]:
feature_importance_df

Feature  Importance
0  TIME_SINCE_FIRST(raw.created_timestamp WHERE h...         2.0
1       MEAN(raw.amount WHERE mode_cleaned = 2)#ever         1.0
2                                         days_on_pg         1.0

In [38]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
3          0.087129         0.062664        0.064196        0.061913    505   
2          0.085470         0.059104        0.060697        0.058530    234   
1          0.047394         0.053663        0.055301        0.053316   4663   

        label_good  label_bad  
Decile                         
3              461         44  
2              214         20  
1             4442        221

In [39]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
3          0.076433         0.064196        0.064196        0.064196    157   
2          0.066946         0.060835        0.061913        0.058530    478   
1          0.043692         0.053660        0.055301        0.053316   3868   

        label_good  label_bad  
Decile                         
3              145         12  
2              446         32  
1             3699        169

In [40]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
3          0.103053         0.062697        0.064196        0.061913    262   
2          0.048077         0.059280        0.060697        0.058530    104   
1          0.059453         0.053683        0.055301        0.053316   2523   

        label_good  label_bad  
Decile                         
3              235         27  
2               99          5  
1             2373        150

In [41]:
xgb_model.save_model('LeoPayu_PG_g15_b24mob_not999.json')
print('Model Saved')

Model Saved
